In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
import os
import pickle

# define the directory where the pickle files are stored
folder_path = '../batched_data_pickle_files/'

# define the filenames for the pickle files
file_names = ['processed_train_data.pkl', 'processed_train_targets.pkl', 'processed_test_data.pkl', 'true_rul.pkl']

# loop through each file and load its contents as arrays
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # read the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    
    # ensure data is a numpy array, if not convert it
    if isinstance(data, np.ndarray):
        globals()[file_name.replace('.pkl', '')] = data
    else:
        globals()[file_name.replace('.pkl', '')] = np.array(data)

print("Processed Train Data Shape:", processed_train_data.shape)
print("Processed Train Target Shape:", processed_train_targets.shape)
print("Processed Test Data Shape:", processed_test_data.shape)
print("True RUL Shape:", true_rul.shape)

Processed Train Data Shape: (17731, 30, 14)
Processed Train Target Shape: (17731,)
Processed Test Data Shape: (100, 30, 14)
True RUL Shape: (100,)


In [6]:
X = processed_train_data
y = processed_train_targets

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

In [7]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

In [15]:
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
from torch.utils.data import DataLoader
import numpy as np

# RMSE loss function
def rmse_loss(predictions, targets):
    return torch.sqrt(nn.MSELoss()(predictions, targets))

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter suggestions by Optuna
    hidden_size = trial.suggest_int('hidden_size', 32, 128, step=32)  # Hidden units
    num_layers = trial.suggest_int('num_layers', 1, 3)  # Number of RNN layers
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Learning rate
    batch_size = trial.suggest_int('batch_size', 32, 128, step=32)  # Batch size
    
    # Initialize the model with the suggested hyperparameters
    model = RNNModel(input_size=X.shape[2], hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    
    # Define loss function and optimizer
    criterion = rmse_loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create DataLoaders with the suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop
    num_epochs = 20  # You can adjust the number of epochs here
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        val_losses = []
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            val_loss = criterion(outputs.squeeze(), batch_y)
            val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)
    
    return avg_val_loss  # Return validation loss

# Initialize Optuna study for hyperparameter optimization
study = optuna.create_study(direction='minimize')  # We want to minimize the validation loss
study.optimize(objective, n_trials=100)  # Run 100 trials for hyperparameter search

# Print the best hyperparameters found by Optuna
print("Best Hyperparameters:", study.best_params)
print("Best Validation RMSE:", study.best_value)

# Extract the best hyperparameters from the best trial
best_trial = study.best_trial
best_hidden_size = best_trial.params['hidden_size']
best_num_layers = best_trial.params['num_layers']
best_learning_rate = best_trial.params['learning_rate']
best_batch_size = best_trial.params['batch_size']

# Instantiate the model with the best hyperparameters
best_model = RNNModel(input_size=X.shape[2], hidden_size=best_hidden_size, num_layers=best_num_layers, output_size=1)

# Define optimizer and loss function
optimizer = optim.Adam(best_model.parameters(), lr=best_learning_rate)
criterion = rmse_loss

# Create DataLoaders with the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)

# Final model training with the best hyperparameters
num_epochs = 20  # You can increase this as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

for epoch in range(num_epochs):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        outputs = best_model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()

    # Validation loop
    best_model.eval()
    with torch.no_grad():
        val_losses = []
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            outputs = best_model(batch_X)
            val_loss = criterion(outputs.squeeze(), batch_y)
            val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_rnn_model_optuna.pth')
print("Best RNN model saved to 'best_rnn_model_optuna.pth'")

# Evaluate on test data (optional)
best_model.eval()
test_tensor = torch.FloatTensor(processed_test_data).to(device)
with torch.no_grad():
    test_predictions = best_model(test_tensor).numpy().squeeze()

# Calculate RMSE on test data
test_rmse = np.sqrt(np.mean((test_predictions - true_rul)**2))
print(f'Test RMSE: {test_rmse:.4f}')



[I 2024-11-21 07:14:31,254] A new study created in memory with name: no-name-bfeaf0e5-a313-4091-b218-438f36625d08
/tmp/ipykernel_9994/934372295.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Learning rate
[I 2024-11-21 07:15:39,638] Trial 0 finished with value: 18.26929014223116 and parameters: {'hidden_size': 128, 'num_layers': 1, 'learning_rate': 0.0034954850746481783, 'batch_size': 32}. Best is trial 0 with value: 18.26929014223116.
[I 2024-11-21 07:19:11,973] Trial 1 finished with value: 81.99670818873814 and parameters: {'hidden_size': 128, 'num_layers': 3, 'learning_rate': 1.2913086955327018e-05, 'batch_size': 128}. Best is trial 0 with value: 18.26929014223116.
[I 2024-11-21 07:20:07,505] Trial 2 finished with value: 81.99715151105609 and para

Best Hyperparameters: {'hidden_size': 32, 'num_layers': 2, 'learning_rate': 0.0011843203255513526, 'batch_size': 32}
Best Validation RMSE: 12.834748358339876
Epoch [1/20], Validation Loss: 72.3708
Epoch [2/20], Validation Loss: 65.1407
Epoch [3/20], Validation Loss: 56.8781
Epoch [4/20], Validation Loss: 48.4448
Epoch [5/20], Validation Loss: 42.7630
Epoch [6/20], Validation Loss: 41.8939
Epoch [7/20], Validation Loss: 41.7637
Epoch [8/20], Validation Loss: 41.7608
Epoch [9/20], Validation Loss: 40.0594
Epoch [10/20], Validation Loss: 20.2189
Epoch [11/20], Validation Loss: 18.4146
Epoch [12/20], Validation Loss: 18.4378
Epoch [13/20], Validation Loss: 18.0692
Epoch [14/20], Validation Loss: 18.1642
Epoch [15/20], Validation Loss: 17.8939
Epoch [16/20], Validation Loss: 17.6741
Epoch [17/20], Validation Loss: 17.7719
Epoch [18/20], Validation Loss: 17.1611
Epoch [19/20], Validation Loss: 16.8547
Epoch [20/20], Validation Loss: 16.9993
Best RNN model saved to 'best_rnn_model_optuna.pth'

In [22]:
# Extract the best hyperparameters from the best trial
best_trial = study.best_trial
best_hidden_size = best_trial.params['hidden_size']
best_num_layers = best_trial.params['num_layers']
best_learning_rate = best_trial.params['learning_rate']
best_batch_size = best_trial.params['batch_size']

# Instantiate the model with the best hyperparameters
best_model = RNNModel(input_size=X.shape[2], hidden_size=best_hidden_size, num_layers=best_num_layers, output_size=1)

# Define optimizer and loss function
optimizer = optim.Adam(best_model.parameters(), lr=best_learning_rate)
criterion = rmse_loss

# Create DataLoaders with the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)

# Final model training with the best hyperparameters
num_epochs = 30  # You can increase this as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

for epoch in range(num_epochs):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        outputs = best_model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()

    # Validation loop
    best_model.eval()
    with torch.no_grad():
        val_losses = []
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            outputs = best_model(batch_X)
            val_loss = criterion(outputs.squeeze(), batch_y)
            val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

# Save the best model to a file
torch.save(best_model.state_dict(), 'best_rnn_model_optuna.pth')
print("Best RNN model saved to 'best_rnn_model_optuna.pth'")

# Evaluate on test data (optional)
best_model.eval()
test_tensor = torch.FloatTensor(processed_test_data).to(device)
with torch.no_grad():
    test_predictions = best_model(test_tensor).numpy().squeeze()

# Calculate RMSE on test data
test_rmse = np.sqrt(np.mean((test_predictions - true_rul)**2))
print(f'Test RMSE: {test_rmse:.4f}')

Epoch [1/30], Validation Loss: 72.8571
Epoch [2/30], Validation Loss: 60.1122
Epoch [3/30], Validation Loss: 50.5848
Epoch [4/30], Validation Loss: 44.9487
Epoch [5/30], Validation Loss: 42.5667
Epoch [6/30], Validation Loss: 41.8887
Epoch [7/30], Validation Loss: 41.7864
Epoch [8/30], Validation Loss: 22.7499
Epoch [9/30], Validation Loss: 21.5080
Epoch [10/30], Validation Loss: 18.3313
Epoch [11/30], Validation Loss: 17.5140
Epoch [12/30], Validation Loss: 17.0022
Epoch [13/30], Validation Loss: 16.6556
Epoch [14/30], Validation Loss: 16.9519
Epoch [15/30], Validation Loss: 17.5586
Epoch [16/30], Validation Loss: 16.4147
Epoch [17/30], Validation Loss: 16.8203
Epoch [18/30], Validation Loss: 16.9057
Epoch [19/30], Validation Loss: 16.8761
Epoch [20/30], Validation Loss: 16.3438
Epoch [21/30], Validation Loss: 16.4679
Epoch [22/30], Validation Loss: 16.4167
Epoch [23/30], Validation Loss: 16.4471
Epoch [24/30], Validation Loss: 19.6358
Epoch [25/30], Validation Loss: 16.3047
Epoch [26